## RFP to Proposal Project

Import Libraries

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import load_dotenv
import re 
import time
from copy import deepcopy
from PyPDF2 import PdfReader
from docx import Document
import openai
import json
import ast
import pandas as pd

# 최신 LangChain 기준
# from langchain_core.documents import Document
# from langchain.chat_models import ChatOpenAI
# from langchain.chains import LLMChain
# import gradio as gr

Parameters

In [37]:
# Load environment variables in a file called .env

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
serp_api_key = os.getenv("SERP_API_KEY")
perplexity_api_key = os.getenv("PEPLEXITY_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if serp_api_key:
    print(f"serp_api_key exists and begins {serp_api_key[:8]}")
else:
    print("serp_api_key not set")

if perplexity_api_key:
    print(f"perplexity_api_key exists and begins {perplexity_api_key[:8]}")
else:
    print("perplexity_api_key not set")

# GPT 모델 선언
openai = OpenAI()
MODEL = 'gpt-5'

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBU
serp_api_key exists and begins c3ee9cec
perplexity_api_key exists and begins pplx-r4f


In [3]:
# PROPOSAL_SLIDE_TEMPLATES = {
#     "cover_page": {
#         "elements": {
#             "Title": "프로젝트의 정식 명칭 (중앙 상단에 크게)",
#             "Subtitle": "고객사명 또는 부제 (타이틀 아래)",
#             "ProjectDate": "작성/제출 일자 (하단 구석)",
#             "PreparedBy": "작성자/제안 주체 (일자 인근)",
#             "Logo": "회사/고객사 로고 (상단 적절 위치)",
#             "DesignBanner": "CI 컬러 강조 바 (상/하단)",
#             "VisualHierarchy": "타이틀 크기/정렬 강조"
#         }
#     },
#     "table_of_contents": {
#         "slide_description": "전체 슬라이드의 구성을 한눈에 제공합니다.",
#         "description": "",
#         "elements": {
#             "Title": "목차 제목 (상단 중앙)",
#             "SectionList": "슬라이드별 주요 제목 리스트 (Bullet, 좌측 정렬)",
#             "SectionDivider": "구분선 등 시각 섹션 구분"
#         }
#     },
#     "executive_summary": {
#         "slide_description": "임원/의사결정자를 위한 핵심 요약, 1~2장",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "프로젝트 핵심 요약",
#             "SummaryPoints": "주요 전략/효과/기간 등 핵심 Bullet",
#             "ClientValuePoints": "효과별 고객 가치, 차별 요약",
#             "QuoteBox": "고객사 인용/키 니즈 강조문",
#             "MetricsOverview": "정량 KPI 요약 박스",
#             "StrategyTriangleView": "전략/성과 구조화 도식",
#             "ValuePillarGraphic": "3대 가치 시각화 (비용, 효율, 위험관리)",
#             "IconVisualCues": "핵심 아이콘 강조"
#         }
#     },
#     "project_understanding": {
#         "slide_description": "프로젝트 필요성 및 배경 설명",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "프로젝트/현황 요약",
#             "KeyObjectives": "고객 주요 목표 Bullet",
#             "BackgroundIssues": "상황/문제점 나열",
#             "StrategicImplications": "비즈니스 영향/리스크 등",
#             "VisualNoteBox": "보충설명, 팁, 인사이트 박스"
#         }
#     },
#     "client_needs_summary": {
#         "slide_description": "고객 요구사항 및 니즈 요약",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "요구 핵심 Summary",
#             "NeedsBulletPoints": "고객 요구사항 리스트",
#             "StrategicDrivers": "전략,정책 등 유발요인",
#             "NeedsMatrix": "요구 항목별 표",
#             "ImplicationColumn": "각 요구의 효과/난이도 등",
#             "RFPQuoteHighlights": "직접 인용 구문",
#             "DesignatedNeedsHeatMap": "중요도/영역별 컬러 맵",
#             "PainToSolutionMappingTable": "Pain-to-Need 매핑표",
#             "OwnershipMatrix": "RACI 기반 책임 매핑"
#         }
#     },
#     "requirement_detail": {
#         "slide_description": "각 요구사항별 상세 설명, 대응방안 등",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "요구 전반 Summary",
#             "RequirementDescription": "각 항목별 상세 설명",
#             "ResponsePlanBox": "대응 방안/실행 요약",
#             "AcceptanceCriteria": "심사/평가 기준 또는 조건",
#             "CategoryTag": "분류(기능/비기능 등)",
#             "DependencyNote": "선행/연계 조건 등",
#             "RequirementTagColoring": "카테고리별 컬러로 시각 구분",
#             "VisualProgressIndicator": "진행률 인디케이터",
#             "PainPointBeforeSummary": "항목별 Pain Point 도입"
#         }
#     },
#     "market_analysis_market_overview": {
#         "slide_description": "시장 크기, 구조, 트렌드 등 시장배경 데이터 제공",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "시장 개요 Summary",
#             "GraphLeft": "시장점유율/성장률 그래프",
#             "TextRight": "시장현황 및 특성 서술",
#             "MarketSegmentationTable": "고객/지역/제품별 시장분류 표",
#             "KeyPlayersBox": "주요 업체 리스트 및 포지셔닝",
#             "InsightBox": "핵심 트렌드/인사이트 강조",
#             "HeatMapVisual": "시장 구조 또는 변동의 히트맵"
#         }
#     },
#     "growth_trend_analysis": {
#         "slide_description": "성장 추이, 드라이버, 전망 등 데이터 기반 분석",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "성장 요약 문장",
#             "GraphLeft": "성장추이/예측 그래프",
#             "TextRight": "해석 및 성장 배경",
#             "GrowthDrivers": "성장 동력 요약 리스트",
#             "ForecastBox": "미래 전망/예측치",
#             "BenchmarkComparison": "국가/기업 비교 표",
#             "OpportunityTags": "성장 기회 영역 태그(GenAI 등)"
#         }
#     },
#     "industry_drivers_challenges": {
#         "slide_description": "성장/저해 요인 비교로 전략 방향 기초 제공",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "변화 요약 또는 시사점 Summary",
#             "DriversList": "산업 동인(성장요소) Left Bullet",
#             "ChallengesList": "장애 요소(저해요인) Right Bullet",
#             "DetailComments": "항목별 보충설명",
#             "StrategicImplicationsBox": "핵심 시사점 강조 박스",
#             "QuoteOrDataSupport": "숫자/전문가 인용 근거"
#         }
#     },
#     "competitive_benchmarking": {
#         "slide_description": "경쟁사와의 정량/정성적 비교",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "경쟁 포지셔닝 요약",
#             "TableMain": {
#                 "description": "핵심 지표 비교 (기능, 가격 등)",
#                 "table_type": "하이라이트 테이블",
#                 "highlight_column": "자사",
#                 "data_table": [["항목", "자사", "K사", "D사"], ["기능수", "12", "10", "9"]]
#             },
#             "BenchmarkTable": {
#                 "description": "세부 Matrix(기업 vs 항목)",
#                 "table_type": "Matrix",
#                 "format": "행:기업, 열:항목"
#             },
#             "CompetitiveInsights": "시장/경쟁사 전략 인사이트",
#             "TextBottom": "요약, 시사점",
#             "DifferentiationBox": "자사만의 차별 요소 강조박스",
#             "CustomerPerception": "고객 만족도/시장 인지도 시각화 (NPS Bar 등)",
#             "PositioningMap": "가격-가치 기반 사분면 도식"
#         }
#     },
#     "swot_analysis": {
#         "slide_description": "SWOT의 각 항목/전략 매트릭스 및 영향도 평가",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "SWOT 분석 Summary",
#             "SWOT-S": "Strength List",
#             "SWOT-W": "Weakness List",
#             "SWOT-O": "Opportunity List",
#             "SWOT-T": "Threat List",
#             "NarrativeSummary": "전략적 시사점",
#             "TOWSMatrix": "SO, ST, WO, WT 매트릭스",
#             "ImpactAssessmentTable": "High/Med/Low 영향도 표",
#             "RadarChartOverlay": "SWOT별 전략 레이더 차트"
#         }
#     },
#     "technical_specifications": {
#         "slide_description": "제안 솔루션의 기술 구조/스펙/정책 상세 설명",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "기술 요약 문장",
#             "TechStackDiagram": "기술스택 구조도(Front/Back 등)",
#             "SystemCompatibility": "플랫폼/브라우저 호환성",
#             "SecurityFeatures": "보안/인증 체계",
#             "PerformanceMetrics": "성능 지표(응답속도 등)",
#             "IntegrationPoints": "외부 시스템 연동방식",
#             "TechRiskMitigation": "기술 리스크/대응 방안",
#             "VersioningPolicy": "버전관리 정책",
#             "TechStackLegend": "스택별 아이콘 표기",
#             "ResiliencyLayerIcon": "이중화/장애복구 구조 강조"
#         }
#     },
#     "service_operation_model": {
#         "slide_description": "운영, 유지보수, 안정성/지속성 설명",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "운영 체계 요약",
#             "OperationOrgChart": "운영조직/지원체계도",
#             "MaintenancePlan": "정기점검/유지보수 계획",
#             "SupportChannels": "고객 지원 채널/운영시간",
#             "CustomerFeedbackLoop": "피드백 수집/반영 절차",
#             "MonitoringTools": "모니터링/알림 시스템",
#             "ServiceLevelAgreementSummary": "SLA 등급 및 기준 요약"
#         }
#     },
#     "compliance_and_governance": {
#         "slide_description": "법적/정책적 준수사항, 거버넌스, 보안/감사체계",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "컴플라이언스, 거버넌스 Summary",
#             "ComplianceList": "준수 기준(법/정책)",
#             "PolicyDiagram": "정책 흐름도, 승인 단계",
#             "DataProtectionMeasures": "데이터 보호/접근제어",
#             "AuditAndMonitoring": "감사/모니터링 체계",
#             "RiskResponseMatrix": "위협 대응 매트릭스",
#             "GovernanceRoles": "거버넌스 R&R 테이블",
#             "RealTimeMonitoringTools": "SIEM / DLP 시스템 요약"
#         }
#     },
#     "client_case_references": {
#         "slide_description": "유사 프로젝트 수행 사례, 신뢰/경험 강조",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "사례 요약",
#             "CaseList": "유사 프로젝트 리스트",
#             "VisualHighlights": "대표 사례 이미지/그래프",
#             "SuccessIndicators": "성과 요약 지표",
#             "ClientTestimonials": "고객사 평가/인용",
#             "IndustryDiversityGraph": "경험 산업군 분포",
#             "CaseImpactSummary": "각 사례의 성과/전략 효과 요약"
#         }
#     },
#     "solution_overview": {
#         "slide_description": "제안 솔루션 전체 가치, 구조 요약, 시각화",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "ValuePropositionBox": {
#                 "description": "핵심 가치 Summary",
#                 "format": ["비용 절감", "신속성", "확장성"]
#             },
#             "SolutionCategoryTag": "유형(SaaS 등) 태그",
#             "SimpleDiagram": {
#                 "description": "Value Chain, Linear 등",
#                 "visual_type": "도식 시각화"
#             },
#             "ArchitectureDiagram": {
#                 "description": "Layer/Block 등 구조도",
#                 "visual_type": "기술 구조"
#             },
#             "KeyModules": {
#                 "description": "주요 기능 요소",
#                 "format": [
#                     {"Module": "Data Collector", "Function": "실시간 수집"},
#                     {"Module": "AI Analyzer", "Function": "패턴 탐지"},
#                     {"Module": "Dashboard", "Function": "시각화"}
#                 ]
#             },
#             "FlexibleIntegrationPoints": {
#                 "description": "연동 API 등 예시",
#                 "examples": ["ERP", "CRM", "Open API", "Cloud"]
#             },
#             "ScalabilityAndSecurity": {
#                 "description": "확장성/보안 체계",
#                 "points": ["모듈화", "RBAC", "데이터 암호화"]
#             },
#             "SolutionDifferentiatorBox": "자사 솔루션만의 차별화 요소 요약"
#         }
#     },
#     "use_case_scenarios": {
#         "slide_description": "주요 활용 시나리오 및 개선효과 도식화",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "시나리오 요약",
#             "ScenarioDiagram": {
#                 "description": "Jouney, Swimlane 등 Flow",
#                 "visual_type": "프로세스/유저 여정"
#             },
#             "NarrativeCases": {
#                 "description": "2~3개 대표 사례(실전 Before/After)",
#                 "format": [
#                     {"Title": "VOC 자동화", "Before": "수작업 2일", "After": "자동 분류, 1시간 내"},
#                     {"Title": "인증 통합", "Before": "이중 로그인", "After": "SSO, 간편화"}
#                 ]
#             },
#             "PersonaIcons": "사용자(실무자/관리자 등) 별 활용 아이콘",
#             "ExpectedOutcomes": "각 시나리오별 핵심 가치(효율, 오류 감소)",
#             "ProcessExceptionHandling": "예외 처리 플로우 및 리스크"
#         }
#     },
#     "strategic_recommendations": {
#         "slide_description": "구체 전략, 실행안, 우선순위, 전략 FLOW 제공",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "전략방향 요약",
#             "BulletPoints": "주요 전략 리스트",
#             "PriorityMap": "우선순위 매트릭스(2x2 등)",
#             "StrategicThemes": "핵심 키 테마",
#             "ActionRoadmap": "전략별 실행 요약",
#             "ClientTailoredValue": "고객 맞춤 전략가치 명시",
#             "ImpactMatrixGrid": "효과/난이도 시각도표",
#             "VisualStrategyFlow": "문제→전략→성과 프로세스화",
#             "StrategyIconSet": "전략별 전용 아이콘"
#         }
#     },
#     "implementation_plan": {
#         "slide_description": "전체 프로젝트 실행계획 및 인력/일정",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "실행로드맵 요약",
#             "TimelineMain": {
#                 "description": "전체 일정(Gantt/Phase)",
#                 "period": "6~12개월",
#                 "visual_type": "Gantt or Phase Timeline"
#             },
#             "Phases": {
#                 "description": "각 단계별 기간/주요내용",
#                 "format": [
#                     {"Phase": "기획/설계", "Duration": "2개월", "Details": "요구/설계"},
#                     {"Phase": "구축/테스트", "Duration": "3개월", "Details": "구현,검증"},
#                     {"Phase": "운영이관", "Duration": "1개월", "Details": "교육,매뉴얼"}
#                 ]
#             },
#             "TextBottom": "리스크/지원 조직/인력 배치",
#             "ImplementationTeam": {
#                 "description": "역할별 투입 인력",
#                 "table_format": [["단계", "역할", "인원"], ["1", "PM", "1"], ["2", "개발", "3"]]
#             },
#             "MilestoneIcons": "Kick-off/UAT 등 주요 아이콘",
#             "ClientCollaborationPoints": "고객 협업, 승인 시점 명시",
#             "SuccessCriteria": "단계별 KPI/완료 기준"
#         }
#     },
#     "timeline_milestones": {
#         "slide_description": "주요 마일스톤 및 일정 시각화",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "일정 전체 요약",
#             "GanttChart": {
#                 "description": "마일스톤 + 일정 도식",
#                 "chart_type": "Gantt/Flow/Roadmap",
#                 "milestones": [
#                     {"Date": "2025.07", "Label": "Kick-off"},
#                     {"Date": "2025.08", "Label": "1차 개발 완료"},
#                     {"Date": "2025.10", "Label": "통합테스트"},
#                     {"Date": "2025.12", "Label": "최종이관"}
#                 ]
#             },
#             "MilestoneIcons": "이정표 상태(착수/완료/승인)",
#             "DependencyNote": "단계간 연계/종속/병행",
#             "ClientCheckpoints": "고객 승인, 회의 등"
#         }
#     },
#     "risk_management_plan": {
#         "slide_description": "리스크 식별 및 대응 전략",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "리스크 관리 요약",
#             "RiskTable": "리스크 리스트(내용/확률/영향)",
#             "MitigationStrategy": "각 리스크별 대응",
#             "RiskPriorityMatrix": "영향-확률 매트릭스(2x2)",
#             "VisualIcons": "주요 리스크 유형 아이콘",
#             "OwnerAssignment": "담당 주체 별 책임"
#         }
#     },
#     "expected_benefits": {
#         "slide_description": "제안 실행시 기대 효과(정성/정량) 표시",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "핵심 효과 요약",
#             "BulletPoints": {
#                 "description": "주요 기대 효과",
#                 "examples": [
#                     "운영 효율",
#                     "매출 증가",
#                     "고객 만족 향상"
#                 ]
#             },
#             "KPIProjection": {
#                 "description": "키 성과지표 변화",
#                 "chart_type": "Line/Bar/Table",
#                 "data": [["항목", "현재", "도입 후"], ["처리시간", "5일", "2일"], ["비용", "100%", "80%"]]
#             },
#             "TangibleVsIntangible": "정량/정성 효과 분리표",
#             "ROIIndicator": "ROI/Payback 구체 수치",
#             "ClientQuoteBox": "기대효과 인용문"
#         }
#     },
#     "investment_budget_estimation": {
#         "slide_description": "예산 총액 및 항목별 세부 내역",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "예산 Summary",
#             "BudgetTable": "총 예산 표(부가세 등 포함)",
#             "CostBreakdown": "항목별 상세 비용(인건비 등)",
#             "OptionalItems": "선택기능 별도 견적",
#             "DiscountPolicyBox": "할인/협상 정책"
#         }
#     },
#     "team_introduction": {
#         "slide_description": "수행팀/핵심인력/조직구조/전문성 표시",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "구성/핵심인력 Summary",
#             "MemberList": "이름, 직책, 경력, 역할",
#             "TeamOrgChart": "조직도(라인, 블록형)",
#             "ExpertiseHighlightIcons": "분야별 시각 아이콘",
#             "ProjectExperienceIndicator": "인력별 수행경험 요약"
#         }
#     },
#     "why_us_differentiation": {
#         "slide_description": "차별 요소/강점 시각화(경쟁사 비교 포함)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "자사 강점 요약",
#             "BulletPoints": {
#                 "description": "주요 차별화 내용",
#                 "examples": [
#                     "B2B/B2G 구축 경험",
#                     "전담 PM/빠른 피드백",
#                     "End-to-End 대응",
#                     "실증 사례 다수"
#                 ]
#             },
#             "ComparisonTable": {
#                 "description": "자사/경쟁사 비교",
#                 "table_format": [
#                     ["항목", "자사", "경쟁사"],
#                     ["경험", "10건 이상", "2~3건"],
#                     ["대응속도", "24h 이내", "72h 이상"]
#                 ]
#             },
#             "DifferentiationIcons": "핵심 강점별 강조 아이콘",
#             "ClientTestimonialHighlight": "고객 직접 평가/인용"
#         }
#     },
#     "closing_summary": {
#         "slide_description": "전체 제안 요약/마무리 메시지/CTA",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "MiddleText": "핵심 한 줄 요약",
#             "FinalCall": {
#                 "description": "종료 CTA",
#                 "options": [
#                     "시범 도입 제안",
#                     "미팅 요청",
#                     "PoC 요청",
#                     "계속 논의 촉구"
#                 ]
#             },
#             "ThankYouVisual": "감사 메시지/이미지",
#             "ContactInfoBox": "이름/이메일/직책/회사",
#             "ReinforcementBanner": "선택 메시지 배너"
#         }
#     },
#     "qna": {
#         "slide_description": "Q&A 인터랙션 및 예상 질의응답 배열",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "EmptySpace": "Q&A 질문 공간",
#             "AnticipatedQuestions": "예상 질문/답변 리스트",
#             "QnABoxIcons": "말풍선 등 시각 아이콘",
#             "EndMessage": "“감사합니다” 마무리 문장"
#         }
#     },
#     "appendix": {
#         "slide_description": "본문 외 참고자료, 통계, 각주 등 집약",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목",
#             "SupportingDetails": "기술 자료, 통계, 수치 등",
#             "Footnotes": "참고 문헌/출처/링크 등 각주",
#             "AppendixNavigationNote": "본문 연결 네비게이션",
#             "DocumentIconList": "자료유형별 아이콘"
#         }
#     },
#     "current_state_snapshot": {
#         "slide_description": "프로젝트 착수 전 고객의 현황, Pain Point, 시스템 환경 등을 요약하여 향후 변화의 필요성을 명확히 합니다.",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예: 현황 진단 Summary)",
#             "CompanyOverviewBox": "고객사의 사업 개요, 조직 구조 핵심 정보 요약",
#             "ProcessOverviewDiagram": "기존 프로세스 개요 (업무 흐름상 문제 발생 위치 시각화)",
#             "PainPointList": "핵심 Pain Point 리스트 (예: 인력 낭비, 시스템 이중화 등)",
#             "CurrentToolMap": "현재 운영 중인 시스템/Tool 구성도 (예: ERP, CRM 등)",
#             "GapAnalysisPoints": "현행 대비 미충족 요건, 개선이 필요한 항목 요약",
#             "HeatmapHighlight": "중요/비효율 영역에 대한 시각적 강조 (컬러 Heatmap 등)"
#         }
#     },
    
#     "stakeholder_analysis_and_communications": {
#         "slide_description": "조직 내외부 이해관계자의 역할, 영향도, 커뮤니케이션 계획을 정리하여 응집력 있는 실행 기반을 마련합니다.",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예: 이해관계자 분석 및 커뮤니케이션 계획)",
#             "StakeholderMap": "이해관계자 그룹 도식 (외부/내부, 높은 영향도/낮은 영향도 분류)",
#             "RolesAndInterestsTable": "이해관계자별 역할, 관심사, 기대 효과 테이블",
#             "CommsStrategyBox": "단계별 커뮤니케이션 전략 (예: 착수/중간/이관 시점)",
#             "EngagementPlanTimeline": "참여 일정 및 접점 시각화 (예: 회의, 승인 포인트)",
#             "RACIOverlay": "계층 간 R(책임자)/A(의사결정)/C/I 역할표시 오버레이"
#         }
#     },
    
#     "data_governance_and_information_strategy": {
#         "slide_description": "데이터 품질 확보, 일관성 유지, 정보 활성화를 위한 정합성 전략 및 통제 체계를 제시합니다.",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예: 데이터 전략 및 정보거버넌스)",
#             "MiddleText": "데이터 전략 핵심 문장",
#             "GovernanceFramework": "Data Governance 조직·정책 체계도 (CIO/보안/관리자 역할 포함)",
#             "DataLifecycleDiagram": "데이터 생성 ~ 폐기에 이르는 흐름도",
#             "StandardizationPolicyBox": "데이터 표준 정책 요약 (용어, 코드, 마스터 등)",
#             "DataQualityCheckList": "정합성 확보 항목 체크리스트",
#             "AccessControlModel": "접근 권한 구조 (RBAC/Attribute 기반 등)",
#             "MetadataRegistrySummary": "메타데이터 관리 체계 및 Data Catalog 구조도"
#         }
#     },
    
#     "change_management_plan": {
#         "slide_description": "새로운 시스템 도입 및 운영방식 전환 시, 변화 저항을 최소화하고 수용도를 높이기 위한 전략적 변화관리 실행안을 제시합니다.",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예: 변화관리 방안)",
#             "MiddleText": "변화관리 전략 요약 문장",
#             "ResistanceRiskList": "예상 Change Resistance 요소들 (예: 업무부담, 학습저항)",
#             "AdoptionPlanPhases": "ADKAR 기반 변화 수용 단계별 계획 (Awareness → Reinforcement)",
#             "TrainingAndEngagementChart": "교육 및 참여 유도 도표",
#             "IncentiveAndRecognition": "변화 수용 강화를 위한 인센티브 요소 정리",
#             "ChampionsNetworkMap": "현업 Champion 조직 및 커뮤니티 맵 (Key Influencer 구분)"
#         }
#     },
    
#     "poc_pilot_strategy": {
#         "slide_description": "본 구축 이전 사전 기술 검증(PoC) 또는 Pilot 프로그램을 통해 고객 우려를 해소하고 성과 예측 기반을 마련합니다.",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예: PoC / Pilot 운영 계획)",
#             "MiddleText": "PoC 전략 요약 문장",
#             "PocScenarioList": "PoC 대상 기능 또는 프로세스 리스트 (2~3개)",
#             "PilotArchitectureDiagram": "Pilot 운영 시 활용될 시스템 구성도",
#             "EvaluationCriteriaTable": "검증 성공/실패 판단 지표 (예: SYSTEM 응답속도 < 1초)",
#             "ExecutionSchedule": "PoC 기간 및 일정표 (예: 2주 실행, 1주 리뷰)",
#             "PilotLimitationNotes": "Pilot의 범위, 한계, 후속 적용 범위 명시",
#             "CustomerInvolvementBox": "고객의 참여 포인트 정리 (검증, 교육, 피드백)"
#         }
#     },
    
#     "post_go_live_support_and_sustainability": {
#         "slide_description": "시스템 도입 이후에도 활발한 활용 및 안정적 운영이 가능하도록 사후 지원 및 지속성 확보 계획을 제시합니다.",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예: 사후지원 및 지속가능성 확보)",
#             "MiddleText": "운영 지속성 요약 문장",
#             "HandoverChecklist": "운영 이관 시 고려사항 체크리스트",
#             "SlaSummaryTable": "서비스 수준협약(SLA) 요약 테이블",
#             "ScheduledMaintenancePlan": "정기 점검 및 유지보수 계획",
#             "SecondPhaseEnhancementBox": "1차 종료 후 고도화 방안 or 옵션 기능",
#             "KnowledgeTransferTimeline": "교육 및 문서 제공(운영매뉴얼, 관리자 교육) 일정표",
#             "SustainabilityIndicators": "성과 유지 위한 주요 지표 (사용률, 이탈률 등)"
#         }
#     }
# }


In [4]:
# # ppt template으로 실제 제안서에 사용될 정도의 quality가 나올수 있도록 고도화 필요
# # 같은 슬라이드라도 slide_description에 따라 제안서 mapping 필요 
# # ex). requirement_detail_v1, requirement_detail_v2

# PROPOSAL_SLIDE_TEMPLATES = {
#     "cover_page": {
#         "elements": {
#             "Title": "프로젝트의 정식 명칭 (중앙 상단에 크게 배치) (예시)",
#             "Subtitle": "고객사명 또는 부제 설명 (Title 아래 위치) (예시)",
#             "ProjectDate": "제안서 작성 또는 제출 일자 (하단 우측 또는 좌측 구석에 배치) (예시)",
#             "PreparedBy": "작성자 또는 제안 주체 (ProjectDate 인근 또는 하단 중앙) (예시)",
#             "Logo": "회사 또는 고객사 로고 (우상단 또는 좌상단에 적절히 배치) (예시)"
#         },
#     },
#     "table_of_contents": {
#         "slide_description": "전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "목차 제목 (상단 중앙) (예시)",
#             "SectionList": "슬라이드별 주요 제목 리스트 (Bullet 형식으로 왼쪽 정렬) (예시)"
#         },
#     },
#     "executive_summary": {
#         "slide_description": "제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예: 제안 개요, Executive Summary) (예시)",
#             "MiddleText": "프로젝트 요약 또는 핵심 문장 (제목 아래 강조 박스) (예시)",
#             "SummaryPoints": "핵심 제안 내용 Bullet (전략, 기대 효과, 기간, 투자 규모 등) (예시)",
#             "ClientValuePoints": "SummaryPoints 별 고객에게 제공되는 핵심 가치 또는 차별화된 이점 요약 (예시)",
#             "QuoteBox": "고객사 경영진의 인용문 또는 고객의 핵심 니즈를 대변하는 문장 (슬라이드 하단 강조용) (예시)",
#             "MetricsOverview": "정량 지표 요약 박스 (예: RFP의 KPI 등) (예시)"
#         },
#         "needs_research": []
#     },
#     "project_understanding": {
#         "slide_description": "프로젝트의 필요성과 배경을 설명하며, 고객의 상황과 과제를 명확히 인식하고 있다는 점을 전달합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (상단 중앙) (예시)",
#             "MiddleText": "프로젝트 요약 또는 핵심 문장 (제목 아래 강조 박스) (예시)",
#             "KeyObjectives": "고객의 주요 목표 목록 (왼쪽 열에 Bullet 형식) (예시)",
#             "BackgroundIssues": "해결하고자 하는 문제 또는 현재 상황 (오른쪽 열에 Bullet 형식) (예시)",
#             "StrategicImplications": "현 상황이 비즈니스에 미치는 영향 (예: 고객경험 저하, 비용 상승 등) (예시)"
#         },
#         "needs_research": []
#     },
#     "client_needs_summary": {
#         "slide_description": "고객의 구체적인 요구사항을 명확하게 정리해 실무적 방향성과 대응의 기준을 제시합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (상단 중앙) (예시)",
#             "MiddleText": "요구사항 요약 핵심 문장 (예시)",
#             "NeedsBulletPoints": "고객 니즈 요약 문장 리스트 (예시)",
#             "StrategicDrivers": "고객 요구를 유발한 내부/외부 전략 요인 (ex. 정책 변화, DX 전략 등)",
#             "NeedsMatrix": "요구사항을 정리한 표 (예시)",
#             "ImplicationColumn": "NeedsMatrix 내 각 요구에 따른 비즈니스적 영향 또는 구현 난이도 칼럼 추가 (예시)",
#             "RFPQuoteHighlights": "RFP 문서 내 직접적인 인용문 (ex. 운영 효율성을 20% 이상 개선해야 함) 필요시"
#         },
#         "needs_research": []
#     },
#     "requirement_detail": {
#         "slide_description": "프로젝트 또는 제안의 성공을 위해 반드시 충족되어야 할 요구사항(client_needs_summary의 NeedsBulletPoints)을 각 항목별로 명확하게 설명하고 제안하는 슬라이드입니다",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (상단 중앙) (예시)",
#             "MiddleText": "요구사항 전체를 아우르는 요약 문장 또는 슬라이드의 핵심 목적 (예시)",
#             "RequirementDescription": "한가지 요구사항에 대한 상세 설명 (예시, 각 Bullet 옆에 간략히 설명 또는 하단에 별도 박스 형태로 추가)",
#             "ResponsePlanBox": "각 요구사항에 대해 우리 측이 어떻게 대응하고 충족할 것인지 상세히 제시(필수)",
#             "AcceptanceCriteria": "각 요구사항이 충족되었음을 판단할 수 있는 구체적 기준 또는 예시 (예: 테스트 통과 조건, 성능 수치 등)",
#             "CategoryTag": "요구사항의 유형 분류 태그 (예: 기능적, 비기능적, 비즈니스, 기술 등)",
#             "DependencyNote": "해당 요구사항과 연관된 다른 요구사항 또는 선행 조건에 대한 설명 (필요시)",
#             "SolutionApproachDiagram": "요구사항을 실현하기 위한 구체적인 접근 방법 도식화 (예: 3단계 접근, waterfall vs agile 등)",
#             "KeyTasksAndDeliverables": "제안 내용에 대응하는 주요 수행 Task와 산출물 목록 (표 형식 또는 아이콘 목록 추천)",
#             "EffortEstimationBox": "요구 충족을 위한 소요 기간/투입 인력 추정 (예: 3주, 디자이너 1명, 개발 2명 등)",
#             "RiskAndMitigationBox": "요구사항 수행 중 예상되는 리스크와 대응 방안 간략 정리",
#             "ToolOrTechnologyUsed": "본 요구를 달성하는 데 사용되는 대표 기술, 툴, 프레임워크 (예: ChatGPT API, Tableau, OpenID 등)",
#             "StepByStepActionPlan": "구체적 실행 절차 요약 (예: 계획 → 설계 → 구현 → 검증 → 완료)"
#         },
#         "needs_research": []
#     },
#     "market_analysis_market_overview": {
#         "slide_description": "해당 산업의 크기, 구조, 주요 트렌드를 데이터와 함께 제시해 전체 시장 배경을 설명합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "시장 개요 요약 문장 (예시)",
#             "GraphLeft": "시장 관련 그래프 (예: 점유율, 성장률 등) (예시)",
#             "TextRight": "시장 현황 및 주요 특징 설명 텍스트 (예시)",
#             "MarketSegmentationTable": "시장 세분화 표 (예: 고객 유형, 지역, 제품군 등) (예시)",
#             "KeyPlayersBox": "주요 기업 리스트 및 시장 내 포지셔닝 설명 (예시)",
#             "InsightBox": "시장 전체에서 가장 주목할 트렌드 1~2가지 강조 인사이트 (아이콘/강조 색 박스) (예시)"
#         },
#         "needs_research": []
#     },
#     "growth_trend_analysis": {
#         "slide_description": "시장 혹은 기술의 성장세를 데이터 기반으로 시각화하고 그 시사점을 제시합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "성장 트렌드 핵심 요약 문장 (예시)",
#             "GraphLeft": "성장 추이 그래프 (선형, 누적 등) (예시)",
#             "TextRight": "그래프 해석 및 성장 요인 서술 (예시)",
#             "GrowthDrivers": "성장 동력 요약 리스트 (예: 기술 혁신, 정책 수혜 등) (예시)",
#             "ForecastBox": "향후 전망 수치 (예: CAGR, 2028년까지 3배 성장 등) 강조 박스 (예시)",
#             "BenchmarkComparison": "동일 산업 내 경쟁국 또는 글로벌 평균과 비교 분석 그래프/표 (예시)",
#             "OpportunityTags": "고성장 영역에 대한 기회 태그 (예: GenAI, SaaS, Micro-mobility 등) (예시)"
#         },
#         "needs_research": []
#     },
#     "industry_drivers_challenges": {
#         "slide_description": "산업 내 성장 요인과 저해 요인을 대조해 전략 방향 설정의 기초로 삼습니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "산업 변화 요약 또는 주요 시사점 (예시)",
#             "DriversList": "산업 동인 목록 (왼쪽 컬럼 Bullet 형식) (예시)",
#             "ChallengesList": "산업 장애 요소 목록 (오른쪽 컬럼 Bullet 형식) (예시)",
#             "DetailComments": "각 항목에 대한 보충 설명 또는 전략적 인사이트 (예시)",
#             "StrategicImplicationsBox": "전략 수립 시 반드시 고려해야 할 핵심 인사이트 정리 (박스 형태) (예시)",
#             "QuoteOrDataSupport": "각 요인에 대한 정량적 근거 또는 고객/전문가 인용문 (예시)"
#         },
#         "needs_research": []
#     },
#     "competitive_benchmarking": {
#         "slide_description": "경쟁사 대비 자사의 상대적 위치와 우위를 정량적·정성적으로 설명합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "경쟁 포지셔닝 요약 (예시)",
#             "TableMain": {
#                 "description": "핵심 비교 지표 (예: 가격, 기능 수, 고객 수 등) (예시)",
#                 "table_type": "하이라이트 테이블 (예시)",
#                 "highlight_column": "자사 기준 (예시)",
#                 "data_table": [["항목", "자사", "경쟁사(KPMG)", "경쟁사(딜로이트)"], ["기능 수", "12", "10", "9"]]
#             },
#             "BenchmarkTable": {
#                 "description": "세부 항목 비교 (세부 제품/서비스 기준) (예시)",
#                 "table_type": "Matrix 테이블 (예시)",
#                 "format": "행: 기업 / 열: 항목 (예: UX, 확장성, 기술 스택 등) (예시)"
#             },
#             "CompetitiveInsights": "경쟁사 전략 및 고객 시사점 도출 (예시)",
#             "TextBottom": "요약 또는 인사이트 도출 문장 (예시)",
#             "DifferentiationBox": "자사의 차별화 요소를 강조하는 요약 박스 (예: Only, First, Unique 등) (예시)",
#             "CustomerPerception": "시장 내 고객의 인지도 또는 만족도 관련 데이터 시각화 (설문 or NPS) (예시)"
#         },
#         "needs_research": []
#     },
#     "swot_analysis": {
#         "slide_description": "내부·외부 환경 분석을 통해 전략적 포지션을 진단하고 방향성을 제시합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "SWOT 분석 요약 문장 (예시)",
#             "SWOT-S": "Strength 요소 리스트 (예시)",
#             "SWOT-W": "Weakness 요소 리스트 (예시)",
#             "SWOT-O": "Opportunity 요소 리스트 (예시)",
#             "SWOT-T": "Threat 요소 리스트 (예시)",
#             "NarrativeSummary": "요약 설명 또는 전략적 시사점 (예시)",
#             "TOWSMatrix": "SWOT 기반 전략 매트릭스 (예: SO, ST, WO, WT 전략 각각 예시) (예시)",
#             "ImpactAssessmentTable": "각 요소의 영향도 및 긴급도 평가 (High/Medium/Low) (예시)"
#         },
#         "needs_research": []
#     },
#     "technical_specifications": {
#         "slide_description": "제안 솔루션의 기술적 구조, 사용 기술, 플랫폼 등을 상세히 설명합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "기술 사양 요약 (예시)",
#             "TechStackDiagram": "기술 스택 구조도 (Front-End, Back-End, DB 등) (예시)",
#             "SystemCompatibility": "플랫폼/브라우저 호환성 설명 (예시)",
#             "SecurityFeatures": "보안 및 인증 체계 요약 (예시)",
#             "PerformanceMetrics": "성능 지표 (예시)",
#             "IntegrationPoints": "외부 시스템과의 연동 포인트 및 방식 (API/FTP 등) (예시)",
#             "TechRiskMitigation": "기술 리스크 및 대응 방안 (예시)",
#             "VersioningPolicy": "버전 관리 정책 및 릴리즈 주기 (예시)"
#         },
#         "needs_research": []
#     },
#     "service_operation_model": {
#         "slide_description": "서비스 운영 및 유지보수 체계를 설명하여 안정성 및 지속 가능성을 강조합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "운영 모델 요약 (예시)",
#             "OperationOrgChart": "운영 조직도 또는 지원 체계도 (예시)",
#             "MaintenancePlan": "유지보수 및 정기점검 계획 (예시)",
#             "SupportChannels": "지원 채널 및 운영 시간 정보 (예시)",
#             "CustomerFeedbackLoop": "고객 피드백 수집 및 반영 절차 (예시)",
#             "MonitoringTools": "사용 중인 모니터링/알림 시스템 (예: Grafana, Slack Alert 등) (예시)"
#         },
#         "needs_research": []
#     },
#     "compliance_and_governance": {
#         "slide_description": "법적/정책적 준수 사항 및 거버넌스 체계를 설명합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "컴플라이언스 및 거버넌스 요약 (예시)",
#             "ComplianceList": "준수 기준 리스트 (예시)",
#             "PolicyDiagram": "보안/정책 흐름도 (예시)",
#             "DataProtectionMeasures": "데이터 보호 및 접근제어 조치 (예시)",
#             "AuditAndMonitoring": "감사 및 모니터링 체계 요약 (예시)",
#             "RiskResponseMatrix": "위협/리스크 발생 시 대응 전략 매트릭스 (예: 탐지 → 대응 → 복구) (예시)",
#             "GovernanceRoles": "거버넌스 참여 부서 및 R&R 요약 테이블 (예시)",
#             "RealTimeMonitoringTools": "SIEM, DLP 등 활용 중인 실시간 모니터링 시스템 요약 (예시)"
#         },
#         "needs_research": []
#     },
#     "client_case_references": {
#         "slide_description": "과거 수행한 유사 프로젝트 사례를 통해 신뢰성과 경험을 강조합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "사례 요약 (예시)",
#             "CaseList": "유사 프로젝트 리스트 (고객사, 수행 내용, 성과 등) (예시)",
#             "VisualHighlights": "대표 사례 이미지 또는 그래프 (예시)",
#             "SuccessIndicators": "주요 성과 지표 요약 (예시)",
#             "ClientTestimonials": "고객사의 평가 또는 인용문 (예시)",
#             "IndustryDiversityGraph": "수행 경험 산업군 분포 (도넛 차트 등) (예시)",
#             "CaseImpactSummary": "각 사례의 전략적 효과 요약 (예: 고객만족도 증가, 보안 리스크 감소 등) (예시)"
#         },
#         "needs_research": []
#     },
#     "solution_overview": {
#         "slide_description": "제안 솔루션의 핵심 가치와 구조를 시각화하고 간명하게 전달합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "ValuePropositionBox": {
#                 "description": "고객에게 제공되는 3대 핵심 가치 또는 효익 (ex: 비용 절감, UX 개선, 확장성) (예시)",
#                 "format": ["가치 1 (예시)", "가치 2 (예시)", "가치 3 (예시)"]
#             },
#             "SolutionCategoryTag": "해당 솔루션이 속한 유형 (예: SaaS / 온프레미스 / 하이브리드 / 컨설팅 / 플랫폼 등) (예시)",
#             "SimpleDiagram": {
#                 "description": "End-to-End 흐름 또는 Value Chain 시각화 (예시)",
#                 "visual_type": "Linear / Circular / 3단계 프로세스 등 (예시)"
#             },
#             "ArchitectureDiagram": {
#                 "description": "기술/기능 구조 시각화 (예시)",
#                 "visual_type": "Layer 구조도, Block Diagram, 모듈 구성도 등 (예시)"
#             },
#             "KeyModules": {
#                 "description": "주요 모듈 또는 기능 구성 요소 리스트 (각 항목에 간단한 설명 포함) (예시)",
#                 "format": [
#                     {"Module": "Data Collector (예시)", "Function": "실시간 데이터 수집 (예시)"},
#                     {"Module": "AI Analyzer (예시)", "Function": "패턴 분석 및 이상 탐지 (예시)"},
#                     {"Module": "Dashboard (예시)", "Function": "시각화 및 리포팅 (예시)"}
#                 ]
#             },
#             "FlexibleIntegrationPoints": {
#                 "description": "다양한 외부 시스템과 연동 가능한 인터페이스 또는 API 설명 (예시)",
#                 "examples": ["ERP (예시)", "CRM (예시)", "Open API (예시)", "클라우드 스토리지 등 (예시)"]
#             },
#             "ScalabilityAndSecurity": {
#                 "description": "확장성 및 보안 체계 설명 (예시)",
#                 "points": ["모듈 확장 가능 구조 (예시)", "RBAC 기반 접근 통제 (예시)", "암호화 저장 (예시)"]
#             }
#         },
#         "needs_research": []
#     },
#     "use_case_scenarios": {
#         "slide_description": "솔루션이 실제로 어떻게 작동하고 활용될 수 있는지를 시나리오 기반으로 보여줍니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "시나리오 요약 (예시)",
#             "ScenarioDiagram": {
#                 "description": "워크플로우 또는 사용자 여정 흐름도 (예시)",
#                 "visual_type": "사용자 Journey Map / Swimlane / Flowchart (예시)",
#                 "steps": ["고객 요청 (예시)", "시스템 분석 (예시)", "자동 처리 (예시)", "결과 제공 (예시)"]
#             },
#             "NarrativeCases": {
#                 "description": "시나리오 2~3개 구성 (업무 상황 → 해결 흐름 → 개선 효과) (예시)",
#                 "format": [
#                     {"Title": "고객 VOC 자동 처리 (예시)", "Before": "수작업으로 2일 소요 (예시)", "After": "자동 분류 및 1시간 내 응답 (예시)"},
#                     {"Title": "인증 시스템 통합 (예시)", "Before": "이중 로그인 발생 (예시)", "After": "SSO 기반 사용자 편의성 향상 (예시)"}
#                 ]
#             },
#             "PersonaIcons": "사용자 유형별 아이콘 및 활용 방식 요약 (예: 실무자, 관리자) (예시)",
#             "ExpectedOutcomes": "각 시나리오별 기대 효과 (시간 단축, 오류 감소 등) (예시)",
#             "ProcessExceptionHandling": "예외 상황 처리 플로우 (에러 발생 시 대응 흐름 등) (예시)"
#         },
#         "needs_research": []
#     },
#     "strategic_recommendations": {
#         "slide_description": "고객을 위한 전략적 방향성과 구체 실행안, 우선순위를 제시합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "전략 방향 요약 (예시)",
#             "BulletPoints": "주요 전략 항목 (예시)",
#             "PriorityMap": "우선순위 매트릭스 (예시)",
#             "StrategicThemes": "전략을 묶는 키 테마 (예: 자동화, 통합, 사용자 중심) (예시)",
#             "ActionRoadmap": "각 전략별 실행 계획 한 줄 요약 (예시)",
#             "ClientTailoredValue": "고객 특성 반영 전략 가치 명시 (ex: '공공기관의 규제 대응에 특화') (예시)"
#         },
#         "needs_research": []
#     },
#     "implementation_plan": {
#         "slide_description": "전략 실행을 위한 단계별 로드맵을 구체적으로 설명합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "실행 로드맵 핵심 요약 (예시)",
#             "TimelineMain": {
#                 "description": "전체 일정 요약 (Gantt 형식 또는 단계별 막대 도식) (예시)",
#                 "period": "6개월 ~ 1년 제안 가능 (예시)",
#                 "visual_type": "Phase Timeline / Roadmap Chart (예시)"
#             },
#             "Phases": {
#                 "description": "각 단계 명칭, 기간, 주요 작업 정리 (예시)",
#                 "format": [
#                     {"Phase": "1단계: 기획/설계 (예시)", "Duration": "~2개월 (예시)", "Details": "요구사항 정의 및 상세 설계 (예시)"},
#                     {"Phase": "2단계: 구축/테스트 (예시)", "Duration": "~3개월 (예시)", "Details": "솔루션 개발, 검증 (예시)"},
#                     {"Phase": "3단계: 운영이관 (예시)", "Duration": "~1개월 (예시)", "Details": "고객 교육 및 운영 매뉴얼 전달 (예시)"}
#                 ]
#             },
#             "TextBottom": "주요 리스크 및 대응 방안, 기술/인력 관련 고려 사항 (예시)",
#             "ImplementationTeam": {
#                 "description": "각 단계별 역할 및 인력 투입 계획 (PM, 개발, QA 등) (예시)",
#                 "table_format": [["단계", "역할", "투입 인원 (예시)"], ["1단계", "PM", "1 (예시)"], ["2단계", "개발자", "3 (예시)"]]
#             },
#             "MilestoneIcons": "각 주요 이벤트에 대한 시각 아이콘 (Kickoff, UAT 등) (예시)",
#             "ClientCollaborationPoints": "고객과의 협업 시점 명시 (승인, 테스트 등) (예시)",
#             "SuccessCriteria": "각 단계별 완료 기준 또는 KPI 정의 (예시)"
#         },
#         "needs_research": []
#     },
#     "timeline_milestones": {
#         "slide_description": "전체 일정에서의 주요 마일스톤을 시각화하여 이해도를 높입니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "일정 요약 또는 주요 포인트 (예시)",
#             "GanttChart": {
#                 "description": "기간별 주요 활동과 마일스톤 도식화 (예시)",
#                 "chart_type": "Gantt / Roadmap / Phase-based Timeline (예시)",
#                 "milestones": [
#                     {"Date": "2025.07 (예시)", "Label": "Kick-off (예시)"},
#                     {"Date": "2025.08 (예시)", "Label": "1차 개발 완료 (예시)"},
#                     {"Date": "2025.10 (예시)", "Label": "통합 테스트 시작 (예시)"},
#                     {"Date": "2025.12 (예시)", "Label": "최종 이관 (예시)"}
#                 ]
#             },
#             "MilestoneIcons": "각 마일스톤에 대한 시각 아이콘 또는 상태 표시 (착수, 승인, 완료 등) (예시)",
#             "DependencyNote": "단계 간 종속 관계 또는 병렬 처리 여부 요약 (예시)",
#             "ClientCheckpoints": "고객 승인/참여 포인트 명시 (UAT, 검토 회의 등) (예시)"
#         },
#         "needs_research": []
#     },
#     "risk_management_plan": {
#         "slide_description": "예상되는 리스크를 식별하고 이에 대한 대응 전략을 제시합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "리스크 관리 개요 (예시)",
#             "RiskTable": "리스크 목록 (내용 | 가능성 | 영향도) (예시)",
#             "MitigationStrategy": "완화 전략 항목 (예시)"
#         },
#         "needs_research": []
#     },
#     "expected_benefits": {
#         "slide_description": "제안서 실행 시 고객이 얻게 될 주요 기대 효과를 정성·정량적으로 표현합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "핵심 기대 효과 요약 (예시)",
#             "BulletPoints": {
#                 "description": "정성적 효과 정리 (예시)",
#                 "examples": [
#                     "운영 프로세스 간소화 및 자동화 (예시)",
#                     "직원 업무 효율 증대 (예시)",
#                     "고객 만족도 향상 (예시)"
#                 ]
#             },
#             "KPIProjection": {
#                 "description": "성과 지표 기반 기대 성과 (예시)",
#                 "chart_type": "Line / Bar / Table (예시)",
#                 "data": [["항목", "현재", "도입 후 (예시)"], ["처리 시간", "5일", "2일 (예시)"], ["운영 비용", "100%", "80% (예시)"]]
#             },
#             "TangibleVsIntangible": "정량 vs 정성 효과 분류 테이블 (예시)",
#             "ROIIndicator": "ROI, Payback 기간 등 주요 재무적 효과 요약 (예시)",
#             "ClientQuoteBox": "고객이 공감할 수 있는 기대 효과 강조 인용문 (예시)"
#         },
#         "needs_research": []
#     },
#     "investment_budget_estimation": {
#         "slide_description": "전체 예산과 각 항목별 비용을 상세히 제시하여 투자 가시성을 제공합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "투자 요약 (예시)",
#             "BudgetTable": "총 예산 테이블 (예시)",
#             "CostBreakdown": "비용 항목별 상세 내용 (예시)"
#         },
#         "needs_research": []
#     },
#     "team_introduction": {
#         "slide_description": "수행팀의 전문성과 역할 분담을 보여줘 신뢰도를 높입니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "팀 구성 및 핵심 인력 요약 (예시)",
#             "MemberList": "핵심 인력 소개 (이름, 역할, 경력 등) (예시)",
#             "TeamOrgChart": "팀 조직도 (예시)"
#         },
#         "needs_research": []
#     },
#     "why_us_differentiation": {
#         "slide_description": "자사만의 차별성과 강점을 강조하여 경쟁사 대비 우위를 설득합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "자사 강점 요약 (예시)",
#             "BulletPoints": {
#                 "description": "차별화된 요소 요약 리스트 (예시)",
#                 "examples": [
#                     "업계 최고 수준의 기술 인력 및 구축 경험 (예시)",
#                     "공공/금융 분야 다수의 실증 사례 (예시)",
#                     "전담 PM 배정 및 신속한 커뮤니케이션 구조 (예시)",
#                     "End-to-End 서비스 제공 (컨설팅 ~ 운영까지) (예시)"
#                 ]
#             },
#             "ComparisonTable": {
#                 "description": "자사 vs 경쟁사 비교 (예시)",
#                 "table_format": [
#                     ["항목", "자사 (예시)", "경쟁사 (예시)"],
#                     ["경험", "10건 이상 유사 구축 (예시)", "2~3건 제한적 (예시)"],
#                     ["대응 속도", "24시간 이내 피드백 (예시)", "72시간 이상 (예시)"]
#                 ]
#             }
#         },
#         "needs_research": []
#     },
#     "closing_summary": {
#         "slide_description": "전체 제안을 요약하며 고객에게 남기고 싶은 핵심 메시지를 강조합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "MiddleText": "핵심 요약 문장 (예시)",
#             "FinalCall": {
#                 "description": "마무리 제안 또는 CTA (예시)",
#                 "options": [
#                     "시범 도입 제안 (예시)",
#                     "추가 미팅 요청 (예시)",
#                     "RFP 연장 논의 (예시)",
#                     "기술 검증(PoC) 요청 등 (예시)"
#                 ]
#             },
#             "ThankYouVisual": "감사 인사와 함께 신뢰를 전달하는 이미지 또는 문구 (예: 함께 성장할 파트너) (예시)",
#             "ContactInfoBox": "담당자 연락처 정보 (이름, 이메일, 직책, 회사 로고 포함) (예시)",
#             "ReinforcementBanner": "슬라이드 하단에 ‘선택의 이유’를 다시 강조하는 요약 배너 (아이콘 또는 문장) (예시)"
#         },
#         "needs_research": []
#     },
#     "qna": {
#         "slide_description": "질의응답을 위한 충분한 공간과 예상 질문 대응 내용을 구성합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "EmptySpace": "Q&A 시각적 공간 (예시)",
#             "AnticipatedQuestions": "예상 질문 및 답변 리스트 (예시)"
#         },
#         "needs_research": []
#     },
#     "appendix": {
#         "slide_description": "통계, 기술 스펙, 참고 문헌 등 본문에 넣기 어려운 자료를 정리합니다. (예시)",
#         "description": "",
#         "elements": {
#             "Title": "슬라이드 제목 (예시)",
#             "SupportingDetails": "기술 자료, 통계 인용 등 (예시)",
#             "Footnotes": "출처, 링크, 각주 등 보충 정보 (예시)"
#         },
#         "needs_research": []
#     }
# }


Functions & Test

In [5]:
def extract_text_from_file(file_path):
    """
    PDF 또는 Word(.docx) 파일에서 텍스트를 추출하고, 줄바꿈 및 띄어쓰기를 정제하는 함수
    """
    from docx import Document
    from PyPDF2 import PdfReader
    ext = os.path.splitext(file_path)[-1].lower()
    text = ""

    if ext == ".pdf":
        try:
            reader = PdfReader(file_path)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        except Exception as e:
            print(f"[PDF 읽기 오류] {e}")

    elif ext == ".docx":
        try:
            doc = Document(file_path)
            for para in doc.paragraphs:
                text += para.text.strip() + "\n"
        except Exception as e:
            print(f"[DOCX 읽기 오류] {e}")

    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 가능합니다.")

    # 후처리: 줄바꿈 정제
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)      # 문장 중간 줄바꿈 제거
    text = re.sub(r"\n{2,}", "\n", text)              # 문단 구분은 하나의 \n으로
    text = re.sub(r"[ \t]{2,}", " ", text)            # 연속 공백 제거

    return text.strip()

In [6]:
# test extract_text_from_file
pdf_path = "DB/RFP/25년 삼성전자 MX 미국 직영 매장 PMO_입찰공고문_F.docx"
rfp_text = extract_text_from_file(pdf_path)
rfp_text[:100]

'프로젝트 배경 및 운영 목표 - 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보가 필요 . 제조업과 달리 직영 리테일은 당사에게는 신규 사업 분야임 . 내/외부 다양한 전문'

In [7]:
def get_user_input(
    rfp_text=None,
    style_selected=None,
    keywords_input=None,
    client_name=None,
    proposal_title=None,
    user_direction=None
):
    """
    사용자 입력 기반 제안서 생성용 입력값 정리 함수
    추후 고객 제안서 데이터 추가등을 통해 고도화 예정
    
    Args:
        rfp_text (str): RFP 원문 텍스트
        style_selected (str): 제안서 스타일 ("격식 있는", "신뢰감 있는" 등)
        keywords_input (str): 강조 키워드 쉼표 구분 (예: "AI, LLM, 효율성")
        client_name (str): 고객사명
        proposal_title (str): 제안서 제목
        user_direction (str): 고객 요청 방향성 
        ***(제안 방안 장성시 해당 방안으로 적용되는지 확인 필요)

    Returns:
        dict: 제안서 생성용 파라미터
    """

    if not rfp_text:
        raise ValueError("⚠️ RFP 텍스트는 필수입니다.")

    style_selected = style_selected or "신뢰감 있는"
    keywords_list = [kw.strip() for kw in (keywords_input or "").split(",") if kw.strip()]
    client_name = client_name or "고객사명 미입력"
    proposal_title = proposal_title or "제안서 제목 미입력"
    user_direction = user_direction or ""

    return {
        "rfp_text": rfp_text,
        "style": style_selected,
        "keywords": keywords_list,
        "client_name": client_name,
        "proposal_title": proposal_title,
        "user_direction": user_direction
    }


In [8]:
# test RFP 텍스트와 일부 값 입력
sample_rfp_text = rfp_text

user_inputs = get_user_input(
    rfp_text=sample_rfp_text,
    style_selected="근거가 있고 formal하게",
    keywords_input="타사 비교, 자체적인, 벤치마크",
    client_name="EY 컨설팅",
    proposal_title="삼성전자 MX 미국 직영 매장 PMO 프로젝트",
    user_direction="현실적으로 운영 가능하고 모두가 납득할만한 근거를 가진 맞춘 설계 필요"
)

# ✅ 출력 확인
for k, v in user_inputs.items():
    print(f"\n🔹 {k}:\n{v if not isinstance(v, str) else v[:50]}")


🔹 rfp_text:
프로젝트 배경 및 운영 목표 - 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보가 

🔹 style:
근거가 있고 formal하게

🔹 keywords:
['타사 비교', '자체적인', '벤치마크']

🔹 client_name:
EY 컨설팅

🔹 proposal_title:
삼성전자 MX 미국 직영 매장 PMO 프로젝트

🔹 user_direction:
현실적으로 운영 가능하고 모두가 납득할만한 근거를 가진 맞춘 설계 필요


In [14]:
def rfp_requirement_check(
    user_inputs: dict,
    model="gpt-4o",
    temperature=0.1
):
    """
    get_user_input 결과 dict를 받아 GPT로 분석하여 PPT 슬라이드 초안 생성

    Args:
        user_inputs (dict): get_user_input 함수에서 반환된 사용자 입력값
        model (str): 사용할 OpenAI 모델
        temperature (float): 생성 자유도

    Returns:
        dict: 슬라이드별 제안서 구성 내용
    """
    rfp_text = user_inputs.get("rfp_text")
    style = user_inputs.get("style", "신뢰감 있는")
    keywords = user_inputs.get("keywords", [])
    client_name = user_inputs.get("client_name", "고객사명 미입력")
    user_direction = user_inputs.get("user_direction", "")

    if not rfp_text or len(rfp_text.strip()) < 30:
        raise ValueError("❗ RFP 원문이 비어 있거나 너무 짧습니다. 실제 RFP를 반드시 입력하세요.")

    # system prompt -> 추후 고객 ppt 데이터 활용
    system_prompt = f"""
    당신은 'EY·맥킨지 등 국내외 최상위 전략 컨설팅 회사의 파트너급 제안서 전문가 AI'입니다.
    <요청사항>
    - 제안 요청 사항을 파악하고, 각 요청 사항에 대해 상세한 제안 방안 제시해주세요.
    - 최대한 많이 제안 요청 사항을 세부적으로 나누어 주세요.
    - 제안 방안에는 RFP내 있는 세부적인 내용을 내포 할수 있도록 해주세요.
    - 마크다운으로 작성하지 마세요
    - 1. RFP에 있는 내용들을 관련이 있다면 제안 요청사항에 세부 항목으로 추가해서 정리하고 진행할 업무에 대해 각각을 제안한다고 생각하고 정리해주세요.
    - 2. 제안 방안을 뒷받침할 질문으로 만들어 주세요 추후 perplexity를 통해 리서치 후 벤치마킹을 하여 고객에게 제공할 예정입니다.
    - 3. 다른 제안을 하기위한 벤치마킹 질문을 만들어주세요.
    - 제안 방안을 실제로 실현 할 수 있기 위해 물어봐야할 질문 리스트를 만들어 주세요 최대한 (상세히). []
    - 질문의 내용은 최대한 상세하게 실제 제안에 유의미하게 작성해주세요.
    안좋은 예시: 프로젝트 관리에서 가장 중요하게 고려해야 할 요소는 무엇인가요?
    좋은 예시: 전자 상품을 파는 리테일 회사의 직영 매장 프로젝트 관리에서 가장 중요하게 고려해야 할 요소는 무엇인가요?
    
    예시 포멧. (질문을 각각 3개씩 만들어야함)
    1. 제안 요청사항: 삼성전자 직영 매장의 성공적인 구축 및 운영 모델확보
    - 제안 방안: 삼성전자의 직영 매장 구축 및 운영을 위한 성공 모델을 확보하기 위해, 제조업과 리테일 업계의 차이점을 분석하고, 내/외부 전문가와의 협업을 통해 빠른 내부 역량 배양을 추진합니다. 또한, 미국 MX 직영 매장의 사례를 벤치마킹하여 글로벌 직영 매장 구축 및 운영 표준 가이드라인을 수립합니다.
    - 제안 방안을 실제로 실현 할 수 있기 위한 질문 3개:
    (1)전자 산업의 제조업과 리테일 업계의 차이점을 분석을 어떻게 하고 차이점은 무엇인가요?
    (2) ...etc
    (3) ...etc
    - 제안 요청사항의 추가 제안 방안 예시를 알기 위한 질문 3개 (벤치마킹 질문): 
    (1)전자 리테일 직영 매장 운영의 모든 영역을 매뉴얼화하기 위해 필요한 세부 항목은 무엇이며, 이를 어떻게 구체적으로 정의할 수 있을까요?
    (2) ...etc
    (3) ...etc
    """

    # user prompt
    user_prompt = f"""
    [RFP 원문]
    {rfp_text}
    
    [고객 방향성/강조]
    {user_direction or '없음'}
    
    [강조 키워드]: {', '.join(keywords) if keywords else '없음'}
    
    위 정보를 바탕으로 제안 요청사항 및 제안 방안을 상세히 정리해주세요.
    """.strip()

    # GPT 호출
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature
    )
    result_text = response.choices[0].message.content.strip()
    bullet_points = [line.strip() for line in result_text.splitlines() if line.strip()]
    return bullet_points

def parse_rfp_response_to_dict(response_lines):
    results = {}
    current_section = {}
    current_key = ""
    section_index = 0
    for line in response_lines:
        line = line.strip()

        if line.startswith(tuple(str(i) + '.' for i in range(1, 100))):
            # 새로운 제안 요청사항 시작
            if current_section:
                results[f"요청사항_{section_index}"] = current_section

            section_index += 1
            current_section = {
                "제안요청사항": line.split(":", 1)[-1].strip(),
                "제안방안": "",
                "실현질문": [],
                "벤치마킹질문": []
            }
            current_key = ""
        
        elif line.startswith("- 제안 방안:"):
            current_section["제안방안"] = line.split(":", 1)[-1].strip()
        
        elif "실현 할 수 있기 위한 질문" in line:
            current_key = "실현질문"
        
        elif "벤치마킹" in line:
            current_key = "벤치마킹질문"
        
        elif line.startswith("(") and current_key:
            current_section[current_key].append(line)

    # 마지막 섹션 저장
    if current_section:
        results[f"요청사항_{section_index}"] = current_section

    return results


In [15]:
rfp_requirement = rfp_requirement_check(user_inputs=user_inputs)
rfp_requirement

['1. 제안 요청사항: 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보',
 '- 제안 방안: 삼성전자의 직영 매장 구축 및 운영을 위한 성공 모델을 확보하기 위해, 제조업과 리테일 업계의 차이점을 분석하고, 내/외부 전문가와의 협업을 통해 빠른 내부 역량 배양을 추진합니다. 또한, 미국 MX 직영 매장의 사례를 벤치마킹하여 글로벌 직영 매장 구축 및 운영 표준 가이드라인을 수립합니다.',
 '- 제안 방안을 실제로 실현 할 수 있기 위한 질문 3개:',
 '(1) 전자 산업의 제조업과 리테일 업계의 차이점을 분석을 어떻게 하고 차이점은 무엇인가요?',
 '(2) 내/외부 전문가와의 협업을 통해 내부 역량을 배양하기 위한 구체적인 계획은 무엇인가요?',
 '(3) 미국 MX 직영 매장의 성공 사례에서 어떤 요소를 벤치마킹하여 글로벌 표준 가이드라인을 수립할 수 있을까요?',
 '- 제안 요청사항의 추가 제안 방안 예시를 알기 위한 질문 3개 (벤치마킹 질문):',
 '(1) 전자 리테일 직영 매장 운영의 모든 영역을 매뉴얼화하기 위해 필요한 세부 항목은 무엇이며, 이를 어떻게 구체적으로 정의할 수 있을까요?',
 '(2) 글로벌 직영 매장 구축을 위한 표준 가이드라인을 수립할 때 고려해야 할 주요 요소는 무엇인가요?',
 '(3) 다른 선진 리테일 기업의 직영 매장 운영 사례에서 배울 수 있는 핵심 인사이트는 무엇인가요?',
 '2. 제안 요청사항: 직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립',
 '- 제안 방안: 매장 운영의 전 영역을 매뉴얼화하고 프로세스를 확립하기 위해, 각 운영 영역에 대한 세부 항목을 정의하고, 이를 기반으로 한 구체적인 매뉴얼을 작성합니다. 또한, 실행 체계 확립을 위한 프로젝트 관리 방안을 마련하여 문서 Tool을 활용한 체계적인 관리가 가능하도록 합니다.',
 '- 제안 방안을 실제로 실현 할 수 있기 위한 질문 3개:',
 '(1) 매장 운영의 전 영역을 매뉴얼화하기 위해 필요한 세부 항목은 무

In [16]:
rfp_dict = parse_rfp_response_to_dict(rfp_requirement)
import pprint
pprint.pprint(rfp_dict, width=160)


{'요청사항_1': {'벤치마킹질문': ['(1) 전자 리테일 직영 매장 운영의 모든 영역을 매뉴얼화하기 위해 필요한 세부 항목은 무엇이며, 이를 어떻게 구체적으로 정의할 수 있을까요?',
                       '(2) 글로벌 직영 매장 구축을 위한 표준 가이드라인을 수립할 때 고려해야 할 주요 요소는 무엇인가요?',
                       '(3) 다른 선진 리테일 기업의 직영 매장 운영 사례에서 배울 수 있는 핵심 인사이트는 무엇인가요?'],
            '실현질문': ['(1) 전자 산업의 제조업과 리테일 업계의 차이점을 분석을 어떻게 하고 차이점은 무엇인가요?', '(2) 내/외부 전문가와의 협업을 통해 내부 역량을 배양하기 위한 구체적인 계획은 무엇인가요?'],
            '제안방안': '삼성전자의 직영 매장 구축 및 운영을 위한 성공 모델을 확보하기 위해, 제조업과 리테일 업계의 차이점을 분석하고, 내/외부 전문가와의 협업을 통해 빠른 내부 역량 배양을 추진합니다. 또한, 미국 MX 직영 매장의 사례를 벤치마킹하여 글로벌 직영 '
                    '매장 구축 및 운영 표준 가이드라인을 수립합니다.',
            '제안요청사항': '삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보'},
 '요청사항_2': {'벤치마킹질문': ['(1) 다른 글로벌 리테일 기업의 매장 운영 매뉴얼화 사례에서 배울 수 있는 핵심 요소는 무엇인가요?',
                       '(2) 매장 운영 프로세스 확립을 위한 선진 사례와 트렌드는 무엇인가요?',
                       '(3) 프로젝트 관리 방안 수립 시, 타사의 성공적인 문서 Tool 활용 사례는 무엇인가요?'],
            '실현질문': ['(1) 매장 운영의 전 영역을 매뉴얼화하기 위해 필요한 세부 항목은 무엇이며, 이를 어떻게 구체적으로 정의할 수 있을까요?',
    

In [19]:
# 🔧 1) 헬퍼: 제안요청사항을 앞에 붙여 자연스러운 질문으로 변환
def make_contextual_question(req_title: str, question: str) -> str:
    req_title = (req_title or "").strip()
    question = (question or "").strip()
    if not req_title:
        return question
    # 필요하면 톤을 바꾸고 싶을 때 이 한 줄만 수정하면 전체 반영됨
    return f"{req_title} 관점에서, {question}"


# 🧾 2) DataFrame (URL 포함) 생성 함수: '질문' 필드도 컨텍스트 반영
def rfp_to_dataframe_with_urls(rfp_dict, answer_dict=None):
    rows = []
    for req_id, req in rfp_dict.items():
        req_title = req.get("제안요청사항", "")
        req_plan = req.get("제안방안", "")
        urls = req.get("urls", [])

        # 실현질문 처리
        실현_질문 = req.get("실현질문", [])
        실현_답변 = answer_dict.get(req_id, {}).get("실현질문_답변", [None] * len(실현_질문)) if answer_dict else [None] * len(실현_질문)
        실현_url = answer_dict.get(req_id, {}).get("실현질문_url", [[] for _ in range(len(실현_질문))]) if answer_dict else [[] for _ in range(len(실현_질문))]

        for question, answer, url_list in zip(실현_질문, 실현_답변, 실현_url):
            contextual_q = make_contextual_question(req_title, question)
            rows.append({
                '요청 ID': req_id,
                '제안요청사항': req_title,
                '제안방안': req_plan,
                '질문유형': '실현질문',
                '질문': contextual_q,   # 👈 컨텍스트 반영
                '답변': answer,
                'url': url_list
            })

        # 벤치마킹질문 처리
        벤치_질문 = req.get("벤치마킹질문", [])
        벤치_답변 = answer_dict.get(req_id, {}).get("벤치마킹질문_답변", [None] * len(벤치_질문)) if answer_dict else [None] * len(벤치_질문)
        벤치_url = answer_dict.get(req_id, {}).get("벤치마킹질문_url", [[] for _ in range(len(벤치_질문))]) if answer_dict else [[] for _ in range(len(벤치_질문))]

        for question, answer, url_list in zip(벤치_질문, 벤치_답변, 벤치_url):
            contextual_q = make_contextual_question(req_title, question)
            rows.append({
                '요청 ID': req_id,
                '제안요청사항': req_title,
                '제안방안': req_plan,
                '질문유형': '벤치마킹질문',
                '질문': contextual_q,   # 👈 컨텍스트 반영
                '답변': answer,
                'url': url_list
            })

    return pd.DataFrame(rows)


# 🧾 3) DataFrame (complete) 생성 함수: 동일하게 '질문' 필드 반영
def rfp_to_dataframe_complete(rfp_dict, answer_dict):
    rows = []
    for req_id, req in rfp_dict.items():
        req_title = req.get("제안요청사항", "제안요청사항 없음")

        # 실현질문 처리
        실현_질문 = req.get("실현질문", [])
        실현_답변 = answer_dict.get(req_id, {}).get("실현질문_답변", [None]*len(실현_질문))
        실현_url = answer_dict.get(req_id, {}).get("실현질문_url", [None]*len(실현_질문))

        for q, a, u in zip(실현_질문, 실현_답변, 실현_url):
            contextual_q = make_contextual_question(req_title, q)
            rows.append({
                "요청 ID": req_id,
                "제안요청사항": req_title,
                "질문유형": "실현질문",
                "질문": contextual_q,  # 👈 컨텍스트 반영
                "답변": a,
                "url": u
            })

        # 벤치마킹질문 처리
        벤치_질문 = req.get("벤치마킹질문", [])
        벤치_답변 = answer_dict.get(req_id, {}).get("벤치마킹질문_답변", [None]*len(벤치_질문))
        벤치_url = answer_dict.get(req_id, {}).get("벤치마킹질문_url", [None]*len(벤치_질문))

        for q, a, u in zip(벤치_질문, 벤치_답변, 벤치_url):
            contextual_q = make_contextual_question(req_title, q)
            rows.append({
                "요청 ID": req_id,
                "제안요청사항": req_title,
                "질문유형": "벤치마킹질문",
                "질문": contextual_q,  # 👈 컨텍스트 반영
                "답변": a,
                "url": u
            })

    return pd.DataFrame(rows)


# 🔍 4) 검색 시에도 컨텍스트가 붙은 질문으로 질의 → 더 정확한 결과
def generate_answer_dict(rfp_dict, api_key, search_mode="perplexity"):
    """
    각 질문에 대해 '제안요청사항'을 앞에 붙인 컨텍스트 질문으로 검색하고,
    답변과 관련 URL 리스트를 받아 answer_dict 형식으로 정리
    """
    answer_dict = {}

    for req_id, section in rfp_dict.items():
        req_title = section.get("제안요청사항", "")
        answer_dict[req_id] = {
            "실현질문_답변": [],
            "실현질문_url": [],
            "벤치마킹질문_답변": [],
            "벤치마킹질문_url": []
        }

        # 실현질문 처리
        for question in section.get("실현질문", []):
            contextual_q = make_contextual_question(req_title, question)
            print(f"[{req_id} - 실현질문] {contextual_q}")
            if search_mode == "serpapi":
                content, urls = search_serpapi_with_url(contextual_q, api_key)
            else:
                content, urls = search_perplexity(contextual_q, api_key)

            answer_dict[req_id]["실현질문_답변"].append(content)
            answer_dict[req_id]["실현질문_url"].append(urls)

        # 벤치마킹질문 처리
        for question in section.get("벤치마킹질문", []):
            contextual_q = make_contextual_question(req_title, question)
            print(f"[{req_id} - 벤치마킹질문] {contextual_q}")
            if search_mode == "serpapi":
                content, urls = search_serpapi_with_url(contextual_q, api_key)
            else:
                content, urls = search_perplexity(contextual_q, api_key)

            answer_dict[req_id]["벤치마킹질문_답변"].append(content)
            answer_dict[req_id]["벤치마킹질문_url"].append(urls)

    return answer_dict


In [20]:
answer_dict = generate_answer_dict(rfp_dict, perplexity_api_key)

[요청사항_1 - 실현질문] 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (1) 전자 산업의 제조업과 리테일 업계의 차이점을 분석을 어떻게 하고 차이점은 무엇인가요?
[요청사항_1 - 실현질문] 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (2) 내/외부 전문가와의 협업을 통해 내부 역량을 배양하기 위한 구체적인 계획은 무엇인가요?
[요청사항_1 - 벤치마킹질문] 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (1) 전자 리테일 직영 매장 운영의 모든 영역을 매뉴얼화하기 위해 필요한 세부 항목은 무엇이며, 이를 어떻게 구체적으로 정의할 수 있을까요?
[요청사항_1 - 벤치마킹질문] 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (2) 글로벌 직영 매장 구축을 위한 표준 가이드라인을 수립할 때 고려해야 할 주요 요소는 무엇인가요?
[요청사항_1 - 벤치마킹질문] 삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (3) 다른 선진 리테일 기업의 직영 매장 운영 사례에서 배울 수 있는 핵심 인사이트는 무엇인가요?
[요청사항_2 - 실현질문] 직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립 관점에서, (1) 매장 운영의 전 영역을 매뉴얼화하기 위해 필요한 세부 항목은 무엇이며, 이를 어떻게 구체적으로 정의할 수 있을까요?
[요청사항_2 - 실현질문] 직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립 관점에서, (2) 매장 운영 프로세스를 확립하기 위한 주요 단계와 고려 사항은 무엇인가요?
[요청사항_2 - 실현질문] 직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립 관점에서, (3) 문서 Tool을 활용한 프로젝트 관리 방안은 어떤 방식으로 구현할 수 있을까요?
[요청사항_2 - 벤치마킹질문] 직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립 관점에서, (1) 다른 글로벌 리테일 기업의 매장 운영 

In [21]:
df = rfp_to_dataframe_with_urls(rfp_dict, answer_dict)
df

,요청 ID,제안요청사항,제안방안,질문유형,질문,답변,url
0,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,"삼성전자의 직영 매장 구축 및 운영을 위한 성공 모델을 확보하기 위해, 제조업과 리...",실현질문,"삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (1) 전자 산업...","전자 산업의 제조업과 리테일(소매) 업계는 목적, 핵심 성공 요인, 운영 구조 등에...",[https://news.tf.co.kr/read/economy/2025627.ht...
1,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,"삼성전자의 직영 매장 구축 및 운영을 위한 성공 모델을 확보하기 위해, 제조업과 리...",실현질문,"삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (2) 내/외부 ...",삼성전자 직영 매장의 성공적 구축 및 운영을 위한 **내‧외부 전문가 협업 기반 내...,[https://news.tf.co.kr/read/economy/2025627.ht...
2,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,"삼성전자의 직영 매장 구축 및 운영을 위한 성공 모델을 확보하기 위해, 제조업과 리...",벤치마킹질문,"삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (1) 전자 리테...",삼성전자 직영 전자 리테일 매장의 성공적인 구축 및 운영을 위해 **모든 운영 영역...,[https://news.tf.co.kr/read/economy/2025627.ht...
3,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,"삼성전자의 직영 매장 구축 및 운영을 위한 성공 모델을 확보하기 위해, 제조업과 리...",벤치마킹질문,"삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (2) 글로벌 직...","삼성전자 글로벌 직영 매장 구축을 위한 **표준 가이드라인**을 수립할 때, 주요 ...",[https://www.chosun.com/special/special_sectio...
4,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,"삼성전자의 직영 매장 구축 및 운영을 위한 성공 모델을 확보하기 위해, 제조업과 리...",벤치마킹질문,"삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보 관점에서, (3) 다른 선진...",다른 선진 리테일 기업의 직영 매장 운영 사례에서 배울 수 있는 핵심 인사이트는 다...,[https://news.tf.co.kr/read/economy/2025627.ht...
5,요청사항_2,직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립,"매장 운영의 전 영역을 매뉴얼화하고 프로세스를 확립하기 위해, 각 운영 영역에 대한...",실현질문,"직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립 관점에서, (1...",직영 매장 운영 전 영역을 매뉴얼화하기 위해서는 매장의 모든 업무와 절차를 **업무...,[https://fastercapital.com/ko/content/%EC%9A%B...
6,요청사항_2,직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립,"매장 운영의 전 영역을 매뉴얼화하고 프로세스를 확립하기 위해, 각 운영 영역에 대한...",실현질문,"직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립 관점에서, (2...","직영 매장 운영 프로세스를 확립하기 위한 주요 단계는 **전 영역 매뉴얼화, 표준화...","[https://kumjac002.tistory.com/10, https://www..."
7,요청사항_2,직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립,"매장 운영의 전 영역을 매뉴얼화하고 프로세스를 확립하기 위해, 각 운영 영역에 대한...",실현질문,"직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립 관점에서, (3...","문서 Tool(예: Notion, Confluence, Google Docs 등)을...",[https://www.docsie.io/blog/ko/articles/what-i...
8,요청사항_2,직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립,"매장 운영의 전 영역을 매뉴얼화하고 프로세스를 확립하기 위해, 각 운영 영역에 대한...",벤치마킹질문,"직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립 관점에서, (1...",글로벌 리테일 기업들이 매장 운영을 매뉴얼화하고 프로세스를 체계화하는 데서 배울 수...,[https://www.shoplworks.com/blog-usecases/fila...
9,요청사항_2,직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립,"매장 운영의 전 영역을 매뉴얼화하고 프로세스를 확립하기 위해, 각 운영 영역에 대한...",벤치마킹질문,"직영 매장 구축 시 매장 운영 전 영역의 매뉴얼화 및 프로세스 확립 관점에서, (2...","직영 매장 운영 프로세스 확립의 선진 사례와 트렌드는 **디지털화, 데이터 기반 의...",[https://fastercapital.com/ko/content/%EC%9A%B...


In [22]:
df = rfp_to_dataframe_complete(rfp_dict, answer_dict)

In [25]:
import pandas as pd
from typing import List, Dict, Any, Tuple, Optional

def _call_llm(prompt: str, model: str = "gpt-4o", temperature: float = 0.3) -> str:
    """LLM 공통 호출 래퍼 (예외 처리 일원화)"""
    try:
        response = openai.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"[GPT 오류] {e}"

def _make_prompt(
    mode: str,
    rfp_text: str,
    요청사항: str,
    기존제안: Optional[str] = None,
    qas_text: Optional[str] = None,
    single_bench: Optional[Tuple[str, str]] = None
) -> str:
    """
    프롬프트 생성:
    - mode == "실현": qas_text(여러 Q/A를 합친 텍스트) + 기존제안 사용
    - mode == "벤치": single_bench = (q, a) 사용
    """
    header = "당신은 EY, 맥킨지 수준의 전략 컨설팅 전문가입니다.\n"
    common_rfp = f"[RFP 요약]\n{rfp_text.strip()}\n\n[제안요청사항]\n{요청사항}\n\n"
    no_md = "- markdown으로 작성하지 마세요"

    if mode == "실현":
        return (
            f"{header}\n"
            "다음 제안요청사항과 기존 제안방안, 그리고 실행을 위한 실현질문 및 답변을 바탕으로, "
            "보다 실행 가능하고 설득력 있는 'updated_제안방안'을 작성해 주세요.\n"
            f"{no_md}\n"
            f"{common_rfp}"
            f"[기존 제안방안]\n{(기존제안 or '').strip()}\n\n"
            f"[실현질문 및 답변]\n{qas_text or ''}"
        ).strip()

    if mode == "벤치" and single_bench:
        q, a = single_bench
        return (
            f"{header}\n"
            "다음 제안요청사항과 RFP 본문, 하나의 벤치마킹 질문과 그에 대한 답변을 바탕으로 "
            "참조 사례와 실행방안을 포함한 'updated_제안방안'을 작성해 주세요.\n"
            f"{no_md}\n"
            f"{common_rfp}"
            "[벤치마킹질문]\n"
            f"Q: {q}\nA: {a}"
        ).strip()

    # 방어적 반환
    return f"{header}\n입력 형식이 올바르지 않습니다."

def generate_updated_plans_per_question(
    rfp_dict: Dict[str, Dict[str, Any]],
    answer_dict: Dict[str, Dict[str, List[Any]]],
    rfp_text: str,
    model: str = "gpt-4o"
) -> pd.DataFrame:
    """
    - 실현질문: 모든 Q/A를 하나로 합쳐 1건의 updated_제안방안 생성
    - 벤치마킹질문: 각 질문마다 updated_제안방안 생성
    """
    rows: List[Dict[str, Any]] = []

    for req_id, section in rfp_dict.items():
        요청사항 = section.get("제안요청사항", "") or ""
        기존제안 = section.get("제안방안", "") or ""

        # ---------- 실현질문: 통합 처리 ----------
        실현_질문들 = section.get("실현질문", []) or []
        실현_답변들 = (answer_dict.get(req_id, {}) or {}).get("실현질문_답변", []) or []
        실현_url들 = (answer_dict.get(req_id, {}) or {}).get(
            "실현질문_url", [[] for _ in 실현_질문들]
        ) or [[] for _ in 실현_질문들]

        if 실현_질문들 and 실현_답변들:
            combined_qas = "\n".join(
                f"{i+1}. Q: {q}\n   A: {a}" for i, (q, a) in enumerate(zip(실현_질문들, 실현_답변들))
            )

            prompt = _make_prompt(
                mode="실현",
                rfp_text=rfp_text,
                요청사항=요청사항,
                기존제안=기존제안,
                qas_text=combined_qas
            )
            result = _call_llm(prompt, model=model, temperature=0.3)

            rows.append({
                "요청 ID": req_id,
                "제안요청사항": 요청사항,
                "질문유형": "실현질문",
                "질문": "\n\n".join(실현_질문들),
                "답변": "\n\n".join(실현_답변들),
                "urls": 실현_url들,
                "updated_제안방안": result
            })

        # ---------- 벤치마킹질문: 개별 처리 ----------
        벤치_질문들 = section.get("벤치마킹질문", []) or []
        벤치_답변들 = (answer_dict.get(req_id, {}) or {}).get("벤치마킹질문_답변", []) or []
        벤치_url들 = (answer_dict.get(req_id, {}) or {}).get(
            "벤치마킹질문_url", [[] for _ in 벤치_질문들]
        ) or [[] for _ in 벤치_질문들]

        for i, (q, a) in enumerate(zip(벤치_질문들, 벤치_답변들)):
            urls = 벤치_url들[i] if i < len(벤치_url들) else []
            prompt = _make_prompt(
                mode="벤치",
                rfp_text=rfp_text,
                요청사항=요청사항,
                single_bench=(q, a)
            )
            result = _call_llm(prompt, model=model, temperature=0.3)

            rows.append({
                "요청 ID": req_id,
                "제안요청사항": 요청사항,
                "질문유형": "벤치마킹질문",
                "질문": q,
                "답변": a,
                "urls": urls,
                "updated_제안방안": result
            })

    return pd.DataFrame(rows)


In [26]:
updated_df = generate_updated_plans_per_question(rfp_dict, answer_dict, rfp_text)


In [27]:
# 엑셀 저장 예시
updated_df.to_excel("updated_제안방안_결과v1.xlsx", index=False)

In [53]:
# -*- coding: utf-8 -*-
"""
SLIM: updated_df -> (요청사항별) 4개 제안 옵션 -> 각 옵션 슬라이드 흐름 -> 저장(JSON/XLSX)
뺀 것: 자동 리서치, 리서치 로그, 검증 파이프라인
남긴 것: 고객이 선택하기 쉬운 옵션 메타(왜/적합/리스크/타임라인) + 슬라이드 텍스트 블록(붙여넣기용)

입력(updated_df 필수 컬럼):
["요청 ID","제안요청사항","질문유형","질문","답변","urls","updated_제안방안"]

출력 파일:
- DB/proposal_result/slim_master_slide_flows.json
- DB/proposal_result/slim_master_slide_flows.xlsx
"""

import os, re, json
import pandas as pd
from typing import Any, Dict, List, Tuple, Optional

# ---- OpenAI Responses API (당신 환경에 맞춤) ----
try:
    openai  # noqa
    MODEL   # noqa
except NameError:
    from openai import OpenAI
    openai = OpenAI()
    MODEL = "gpt-5"

# ---- 유틸 ----
def _resp_text(resp) -> str:
    """OpenAI Responses API 결과에서 텍스트 추출."""
    if hasattr(resp, "output_text") and resp.output_text:
        return resp.output_text
    try:
        parts = resp.output[0].content
        for p in parts:
            if getattr(p, "text", None) and getattr(p.text, "value", None):
                return p.text.value
            if getattr(p, "type", None) == "output_text":
                return p.text
    except Exception:
        pass
    return str(resp)

def _extract_json_block(text: str) -> Optional[Dict[str, Any]]:
    """응답에서 JSON 블록만 추출."""
    if not text:
        return None
    m = re.search(r"```json\s*(\{.*?\})\s*```", text, flags=re.S)
    if m:
        try:
            return json.loads(m.group(1))
        except Exception:
            pass
    start, end = text.find("{"), text.rfind("}")
    if start != -1 and end != -1 and end > start:
        try:
            return json.loads(text[start:end+1])
        except Exception:
            return None
    return None

def _call_llm_json(prompt: str, model: str = None) -> Dict[str, Any]:
    """Responses API 호출 → JSON 파싱 (temperature 미사용)."""
    model = model or MODEL
    try:
        resp = openai.responses.create(model=model, input=prompt)
        text = _resp_text(resp).strip()
    except Exception as e:
        raise RuntimeError(f"LLM 호출 오류: {e}")
    data = _extract_json_block(text)
    if not data:
        raise ValueError("LLM 응답에서 JSON을 추출하지 못했습니다.")
    return data

def _flatten_urls_cell(cell) -> List[str]:
    if cell is None or (isinstance(cell, float) and pd.isna(cell)):
        return []
    if isinstance(cell, (list, tuple)):
        return [str(u) for u in cell if u]
    return [str(cell)]

# ---- 프롬프트: 요청사항 하나당 1회 호출로 4개 옵션 + 슬라이드 전체 생성 ----
def _build_request_deck_prompt(
    req_id: str,
    req_title: str,
    option_seeds: List[Dict[str, Any]],
    max_slides_per_option: int = 6
) -> str:
    """
    option_seeds: [
      {
        "option_no": 1..4,
        "question": "...",
        "answer": "...",
        "updated_plan": "...",
        "urls": ["..."]
      }, ...
    ]
    """
    seeds_txt = []
    for seed in option_seeds:
        urls_block = "\n".join([f"  - {u}" for u in seed.get("urls", [])]) or "  - (없음)"
        seeds_txt.append(
            f"[옵션 {seed['option_no']} 시드]\n"
            f"Q: {seed.get('question','')}\n"
            f"A: {seed.get('answer','')}\n"
            f"Updated Plan: {seed.get('updated_plan','')}\n"
            f"URLs:\n{urls_block}\n"
        )
    seeds_joined = "\n".join(seeds_txt)

    # JSON 스키마(슬림)
    schema = {
        "req_id": req_id,
        "req_title": req_title,
        "overview_slide": {
            "title": "요청사항별 제안 옵션 개요",
            "subtitle": "4개 옵션 비교",
            "purpose": "고객이 옵션을 빠르게 비교·선택할 수 있도록 차이를 명확히 제시",
            "table": {
                "columns": ["옵션", "핵심 전략", "선택 기준", "예상 기간(주)", "주요 리스크"],
                "rows": [["옵션 1", "", "", "", ""]]
            }
        },
        "options": [
            {
                "option_no": 1,
                "option_title": "string (실제 제안명처럼 명확하게)",
                "why_choose": ["선택 이유 3~5개"],
                "fit_signals": ["이 옵션이 맞는 징후 3~5개"],
                "risks": ["핵심 리스크 2~4개"],
                "mitigations": ["리스크 완화책 2~4개"],
                "timeline": [{"phase":"설계","duration_weeks":2},{"phase":"파일럿","duration_weeks":4}],
                "slides": [
                    {
                        "slide_no": f"1..{max_slides_per_option}",
                        "title": "string (실제 PPT 제목)",
                        "subtitle": "string (범위/대상/기간 등 구체화)",
                        "purpose": "이 페이지 핵심 요지 1~2문장",
                        "key_messages": ["실행/지표 중심 3~5개"],
                        "content_draft": "6~10문장, 붙여넣기용 순문장 본문",
                        "paste_blocks": {
                            "title_text": "제목 그대로",
                            "subtitle_text": "부제 그대로",
                            "body_bulleted": ["핵심 메시지용 개조식 텍스트 3~6개"]
                        },
                        "urls": ["위 seed URLs에서만 선택 (없으면 [])"]
                    }
                ]
            }
        ]
    }
    schema_json = json.dumps(schema, ensure_ascii=False, indent=2)

    rules = [
        "- 출력은 반드시 JSON 하나만.",
        "- 각 옵션은 6~12장 슬라이드.",
        "- 각 제안별 슬라이드는 1~3개 이상 차이가 나지 않게 구성.",
        "- 제목/부제목은 실제 PPT 표기처럼 명확하고 구체적으로.",
        "- 정량/사례가 필요하면 seed URLs에서만 인용. 없으면 일반론 금지하고 실행/구조 중심으로.",
        "- paste_blocks는 실제 붙여넣기용 텍스트(개조식 포함)로 채우기.",
    ]

    prompt = (
        "당신은 EY/맥킨지 스타일의 전략 컨설팅 문서 설계 전문가입니다.\n"
        f"요청사항 하나({req_id} - {req_title})에 대해, 아래 시드 4개를 각기 다른 제안 옵션으로 확장하고,\n"
        "각 옵션에 대한 슬라이드 흐름(6~12장)을 설계하세요. 고객이 선택하기 쉽게 옵션 메타도 포함하세요.\n\n"
        f"{seeds_joined}\n"
        "[규칙]\n- " + "\n- ".join(rules) + "\n\n"
        "[JSON 스키마(참고)]\n" + schema_json + "\n\n"
        "JSON만 반환하세요."
    )
    return prompt

# ---- 생성 로직 ----
def build_master_deck_slim(
    updated_df: pd.DataFrame,
    model: str = None,
    max_slides_per_option: int = 6,
    options_per_req: int = 4
) -> Dict[str, Any]:
    """
    한 요청사항당 LLM 1회 호출 → 4개 옵션 + 각 옵션 슬라이드 4~6장.
    research/verification 제거.
    """
    required = {"요청 ID","제안요청사항","질문유형","질문","답변","urls","updated_제안방안"}
    missing = required - set(updated_df.columns)
    if missing:
        raise ValueError(f"updated_df에 필요한 컬럼이 없습니다: {sorted(missing)}")

    sections = []
    for req_id, g in updated_df.groupby("요청 ID", sort=False):
        req_title = str(g["제안요청사항"].iloc[0])
        # 질문 순서대로 4개만 seed로 사용
        seeds = []
        for i, (_, row) in enumerate(g.head(options_per_req).iterrows(), start=1):
            seeds.append({
                "option_no": i,
                "question": str(row["질문"]),
                "answer": str(row["답변"]),
                "updated_plan": str(row["updated_제안방안"]) if pd.notna(row["updated_제안방안"]) else "",
                "urls": _flatten_urls_cell(row["urls"])
            })

        prompt = _build_request_deck_prompt(str(req_id), req_title, seeds, max_slides_per_option)
        data = _call_llm_json(prompt, model=model or MODEL)

        section = {
            "req_id": str(req_id),
            "req_title": req_title,
            "overview_slide": data.get("overview_slide", {}),
            "options": data.get("options", [])
        }
        sections.append(section)

    master = {
        "cover": {
            "title": "제안서(슬림)",
            "subtitle": "요청사항별 4개 제안 옵션과 슬라이드 흐름",
            "fields": {"고객사":"", "작성일":"", "작성팀":""}
        },
        "sections": sections,
        "closing": {
            "title": "다음 단계",
            "bullets": ["옵션 선택 워크숍", "데이터/사전조건 점검", "파일럿 범위 합의 및 킥오프"]
        }
    }
    return master

# ---- DataFrame 변환 & 저장 ----
def deck_to_dataframe_slim(master: Dict[str, Any]) -> pd.DataFrame:
    rows = []
    # 표지
    rows.append({
        "요청 ID":"COVER","요청 제목":"(표지)","옵션번호":"",
        "슬라이드번호":1,"제목":master["cover"]["title"],"부제목":master["cover"]["subtitle"],
        "본문초안":json.dumps(master["cover"]["fields"], ensure_ascii=False),
        "왜_이_옵션":"", "적합_시그널":"", "리스크":"", "완화책":"", "타임라인":"", "URL":""
    })
    for sec in master.get("sections", []):
        req_id, req_title = sec.get("req_id",""), sec.get("req_title","")
        ov = sec.get("overview_slide", {})
        if ov:
            rows.append({
                "요청 ID":req_id,"요청 제목":req_title,"옵션번호":"OVERVIEW",
                "슬라이드번호":0,"제목":ov.get("title","옵션 개요"),"부제목":ov.get("subtitle",""),
                "본문초안":ov.get("purpose",""),
                "왜_이_옵션":"", "적합_시그널":"", "리스크":"", "완화책":"",
                "타임라인":json.dumps(ov.get("table",{}), ensure_ascii=False),
                "URL":""
            })
        for opt in sec.get("options", []):
            opt_no = opt.get("option_no","")
            why = "\n- " + "\n- ".join(opt.get("why_choose", [])) if opt.get("why_choose") else ""
            fit = "\n- " + "\n- ".join(opt.get("fit_signals", [])) if opt.get("fit_signals") else ""
            risk = "\n- " + "\n- ".join(opt.get("risks", [])) if opt.get("risks") else ""
            miti = "\n- " + "\n- ".join(opt.get("mitigations", [])) if opt.get("mitigations") else ""
            timeline = json.dumps(opt.get("timeline", []), ensure_ascii=False)

            # 옵션 요약 행(슬라이드 아님, 메타)
            rows.append({
                "요청 ID":req_id,"요청 제목":req_title,"옵션번호":opt_no,
                "슬라이드번호":"META","제목":opt.get("option_title",""),
                "부제목":"(옵션 요약)","본문초안":"",
                "왜_이_옵션":why,"적합_시그널":fit,"리스크":risk,"완화책":miti,
                "타임라인":timeline,"URL":""
            })

            # 슬라이드들
            for s in opt.get("slides", []):
                urls = "\n".join(s.get("urls", [])) if s.get("urls") else ""
                body = s.get("content_draft","")
                # 붙여넣기 블록도 포함(필요 시)
                pb = s.get("paste_blocks", {})
                if pb:
                    body += "\n\n[붙여넣기]\n- " + "\n- ".join(pb.get("body_bulleted", []))
                rows.append({
                    "요청 ID":req_id,"요청 제목":req_title,"옵션번호":opt_no,
                    "슬라이드번호":s.get("slide_no"),"제목":s.get("title"),"부제목":s.get("subtitle"),
                    "본문초안":body,
                    "왜_이_옵션":"", "적합_시그널":"", "리스크":"", "완화책":"",
                    "타임라인":"", "URL":urls
                })
    # 클로징
    rows.append({
        "요청 ID":"CLOSING","요청 제목":"(마무리)","옵션번호":"",
        "슬라이드번호":1,"제목":master["closing"]["title"],"부제목":"",
        "본문초안":"\n- " + "\n- ".join(master["closing"]["bullets"]),
        "왜_이_옵션":"", "적합_시그널":"", "리스크":"", "완화책":"", "타임라인":"", "URL":""
    })
    return pd.DataFrame(rows)

def run_slim_pipeline(updated_df: pd.DataFrame, out_dir: str = "DB/proposal_result", model: str = None):
    os.makedirs(out_dir, exist_ok=True)
    master = build_master_deck_slim(updated_df, model=model or MODEL)
    df = deck_to_dataframe_slim(master)
    json_path = os.path.join(out_dir, "slim_master_slide_flows.json")
    xlsx_path = os.path.join(out_dir, "slim_master_slide_flows.xlsx")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(master, f, ensure_ascii=False, indent=2)
    df.to_excel(xlsx_path, index=False)
    print(f"[저장 완료] {json_path}")
    print(f"[저장 완료] {xlsx_path}")
    return master, df


In [54]:
master, df = run_slim_pipeline(updated_df, out_dir="DB/proposal_result", model=MODEL)
df.head(20)

[저장 완료] DB/proposal_result\slim_master_slide_flows.json
[저장 완료] DB/proposal_result\slim_master_slide_flows.xlsx


,요청 ID,요청 제목,옵션번호,슬라이드번호,제목,부제목,본문초안,왜_이_옵션,적합_시그널,리스크,완화책,타임라인,URL
0,COVER,(표지),,1,제안서(슬림),요청사항별 4개 제안 옵션과 슬라이드 흐름,"{""고객사"": """", ""작성일"": """", ""작성팀"": """"}",,,,,,
1,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,OVERVIEW,0,요청사항별 제안 옵션 개요,4개 옵션 비교,고객이 옵션을 빠르게 비교·선택할 수 있도록 차이를 명확히 제시,,,,,"{""columns"": [""옵션"", ""핵심 전략"", ""선택 기준"", ""예상 기간(주)...",
2,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,1,META,제조-리테일 격차 해소형 직영 매장 운영 모델 구축,(옵션 요약),,\n- 제조 중심 조직에서 리테일 운영 역량을 빠르게 구축할 수 있다\n- 외부 전...,\n- 현재 매장 운영 KPI가 불안정하거나 점포별 편차가 크다\n- 고객 체험·프...,\n- 현장 변화 저항과 실행 피로도 누적\n- 외부 전문가 의존으로 내재화 지연\...,\n- 현장 리더를 공동설계자로 참여시키는 코디자인 방식 적용\n- 내부 코치 육성...,"[{""phase"": ""진단·설계"", ""duration_weeks"": 4}, {""ph...",
3,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,1,1,제조-리테일 격차 진단과 직영 매장 운영 방향,삼성전자 관점의 핵심 차이점과 시사점 정리,"제조업은 대규모 생산과 품질관리, 공급망 최적화에 강점을 가지며 수직계열화로 경쟁력...",,,,,,https://news.tf.co.kr/read/economy/2025627.htm...
4,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,1,2,현행 역량 진단 프레임워크(4x4),운영·고객경험·디지털·인력 영역별 성숙도 평가 방법,"운영, 고객경험, 디지털, 인력의 네 영역을 4단계 성숙도로 정의한다. 각 영역은 ...",,,,,,https://www.datocms-assets.com/45669/163426966...
5,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,1,3,내/외부 전문가 협업 거버넌스,자문단-프로젝트 셀-현장 코치 삼각 체계,"외부 자문단은 고객경험, 공간, 디지털, 운영 분야로 구성한다. 내부 프로젝트 셀은...",,,,,,https://news.tf.co.kr/read/economy/2025627.htm...
6,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,1,4,파일럿 직영 매장 설계,2개 유형(플래그십/지역거점)에서 8주 실험,플래그십과 지역거점 두 유형을 선정해 병행 파일럿을 수행한다. 각 유형에 맞는 가설...,,,,,,https://www.newsis.com/view/NISX20230307_00022...
7,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,1,5,지식 내재화와 표준화 메커니즘,플레이북-아카데미-인증제 순환 체계,"플레이북은 절차와 체크리스트, 책임, 템플릿을 포함해 실행 친화적으로 구성한다. 아...",,,,,,https://news.tf.co.kr/read/economy/2025627.htm
8,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,1,6,KPI·대시보드 및 확산 로드맵,고객·운영·디지털 KPI와 단계적 확산 계획,"고객, 운영, 디지털의 세 축으로 KPI를 설계한다. 지표는 일·주·월 단위로 추적...",,,,,,https://zdnet.co.kr/view/?no=20230307081420
9,요청사항_1,삼성전자 직영 매장의 성공적인 구축 및 운영 모델 확보,2,META,전사 운영 매뉴얼(SOP) 전면 구축 프로그램,(옵션 요약),,\n- 매장 운영의 변동성을 낮추고 확장성을 높인다\n- 신규·리뉴얼·가맹 확산에 ...,\n- 매장별 동일 이슈가 반복되고 원인 파악이 어렵다\n- 직무별 역할·책임·절차...,\n- 과도한 문서화로 현장 피로 증가\n- 매뉴얼 업데이트 지연으로 현실과 괴리\...,\n- 현장 참여형 CoE가 핵심 항목 중심으로 경량화\n- 분기별 변경관리와 버전...,"[{""phase"": ""현행 진단·범위 확정"", ""duration_weeks"": 3}...",
